In [1]:
import cv2
import numpy as np
import pywt
from skimage.metrics import peak_signal_noise_ratio as psnr, mean_squared_error as mse, structural_similarity as ssim

1. Unsharp Masking

In [2]:
def unsharp_masking(image, sigma=1.0, strength=1.5):
    blurred = cv2.GaussianBlur(image, (0, 0), sigma)
    mask = image - blurred
    sharpened = image + strength * mask
    return np.clip(sharpened, 0, 255)

def process_image(image_path):
    image = cv2.imread(image_path)
    original_shape = image.shape
    channels = cv2.split(image)
    enhanced_channels = []

    for channel in channels:
        if channel.shape[0] % 2 != 0:
            channel = channel[:-1, :]
        if channel.shape[1] % 2 != 0:
            channel = channel[:, :-1]

        coeffs = pywt.dwt2(channel, 'haar')
        LL, (LH, HL, HH) = coeffs

        enhanced_LL = unsharp_masking(LL)

        reconstructed_channel = pywt.idwt2((enhanced_LL, (LH, HL, HH)), 'haar')
        reconstructed_channel = reconstructed_channel[:channel.shape[0], :channel.shape[1]]
        reconstructed_channel = np.uint8(reconstructed_channel)
        enhanced_channels.append(reconstructed_channel)

    reconstructed_image = cv2.merge(enhanced_channels)
    reconstructed_image = cv2.resize(reconstructed_image, (original_shape[1], original_shape[0]))
    return image, reconstructed_image

def calculate_metrics(original, reconstructed):
    original = original.astype(np.float32)
    reconstructed = reconstructed.astype(np.float32)
    psnr_value = psnr(original, reconstructed, data_range=255)
    mse_value = mse(original, reconstructed)
    ssim_value = ssim(original, reconstructed, data_range=255, win_size=3, channel_axis=2)
    return psnr_value, mse_value, ssim_value

image_paths = [
    "C:/Users/Joel Fredrick/Downloads/1.1.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.2.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.3.jpg",
]

for image_path in image_paths:
    original, reconstructed = process_image(image_path)
    psnr_value, mse_value, ssim_value = calculate_metrics(original, reconstructed)

    print(f"Metrics for {image_path}:")
    print(f"PSNR: {psnr_value}")
    print(f"MSE: {mse_value}")
    print(f"SSIM: {ssim_value}")

    cv2.imshow('Original Image', original)
    cv2.imshow('Reconstructed Image - Unsharp Masking', reconstructed)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Metrics for C:/Users/Joel Fredrick/Downloads/1.1.jpg:
PSNR: 15.819857466474673
MSE: 1702.529376437516
SSIM: 0.737873375415802
Metrics for C:/Users/Joel Fredrick/Downloads/1.2.jpg:
PSNR: 10.148105319185845
MSE: 6284.486953032506
SSIM: 0.5582954287528992
Metrics for C:/Users/Joel Fredrick/Downloads/1.3.jpg:
PSNR: 13.767743066546565
MSE: 2730.901614555732
SSIM: 0.5985174775123596


2. High-Pass Filter

In [3]:
def high_pass_filter(image):
    high_pass_kernel = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
    filtered_image = cv2.filter2D(image, -1, high_pass_kernel)
    return filtered_image

def process_image(image_path):
    image = cv2.imread(image_path)
    original_shape = image.shape
    channels = cv2.split(image)
    enhanced_channels = []

    for channel in channels:
        if channel.shape[0] % 2 != 0:
            channel = channel[:-1, :]
        if channel.shape[1] % 2 != 0:
            channel = channel[:, :-1]

        coeffs = pywt.dwt2(channel, 'haar')
        LL, (LH, HL, HH) = coeffs

        filtered_LL = high_pass_filter(LL)
        enhanced_LL = LL + filtered_LL

        reconstructed_channel = pywt.idwt2((enhanced_LL, (LH, HL, HH)), 'haar')
        reconstructed_channel = reconstructed_channel[:channel.shape[0], :channel.shape[1]]
        reconstructed_channel = np.uint8(reconstructed_channel)
        enhanced_channels.append(reconstructed_channel)

    reconstructed_image = cv2.merge(enhanced_channels)
    reconstructed_image = cv2.resize(reconstructed_image, (original_shape[1], original_shape[0]))
    return image, reconstructed_image

def calculate_metrics(original, reconstructed):
    original = original.astype(np.float32)
    reconstructed = reconstructed.astype(np.float32)
    psnr_value = psnr(original, reconstructed, data_range=255)
    mse_value = mse(original, reconstructed)
    ssim_value = ssim(original, reconstructed, data_range=255, win_size=3, channel_axis=2)
    return psnr_value, mse_value, ssim_value

image_paths = [
    "C:/Users/Joel Fredrick/Downloads/1.1.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.2.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.3.jpg",
]

for image_path in image_paths:
    original, reconstructed = process_image(image_path)
    psnr_value, mse_value, ssim_value = calculate_metrics(original, reconstructed)

    print(f"Metrics for {image_path}:")
    print(f"PSNR: {psnr_value}")
    print(f"MSE: {mse_value}")
    print(f"SSIM: {ssim_value}")

    cv2.imshow('Original Image', original)
    cv2.imshow('Reconstructed Image - High-Pass Filtering', reconstructed)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Metrics for C:/Users/Joel Fredrick/Downloads/1.1.jpg:
PSNR: 10.746982956147079
MSE: 5474.974052304285
SSIM: 0.40413832664489746
Metrics for C:/Users/Joel Fredrick/Downloads/1.2.jpg:
PSNR: 8.907106807697085
MSE: 8363.146029494728
SSIM: 0.20899374783039093
Metrics for C:/Users/Joel Fredrick/Downloads/1.3.jpg:
PSNR: 12.46598109103047
MSE: 3685.379805688041
SSIM: 0.2843369245529175


3. Contrast Enhancement

In [4]:
def contrast_enhancement(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(np.uint8(image))
    return enhanced_image

def process_image(image_path):
    image = cv2.imread(image_path)
    original_shape = image.shape
    channels = cv2.split(image)
    enhanced_channels = []

    for channel in channels:
        if channel.shape[0] % 2 != 0:
            channel = channel[:-1, :]
        if channel.shape[1] % 2 != 0:
            channel = channel[:, :-1]

        coeffs = pywt.dwt2(channel, 'haar')
        LL, (LH, HL, HH) = coeffs

        enhanced_LL = contrast_enhancement(LL)

        reconstructed_channel = pywt.idwt2((enhanced_LL, (LH, HL, HH)), 'haar')
        reconstructed_channel = reconstructed_channel[:channel.shape[0], :channel.shape[1]]
        reconstructed_channel = np.uint8(reconstructed_channel)
        enhanced_channels.append(reconstructed_channel)

    reconstructed_image = cv2.merge(enhanced_channels)
    reconstructed_image = cv2.resize(reconstructed_image, (original_shape[1], original_shape[0]))
    return image, reconstructed_image

def calculate_metrics(original, reconstructed):
    original = original.astype(np.float32)
    reconstructed = reconstructed.astype(np.float32)
    psnr_value = psnr(original, reconstructed, data_range=255)
    mse_value = mse(original, reconstructed)
    ssim_value = ssim(original, reconstructed, data_range=255, win_size=3, channel_axis=2)
    return psnr_value, mse_value, ssim_value

image_paths = [
    "C:/Users/Joel Fredrick/Downloads/1.1.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.2.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.3.jpg",
]

for image_path in image_paths:
    original, reconstructed = process_image(image_path)
    psnr_value, mse_value, ssim_value = calculate_metrics(original, reconstructed)

    print(f"Metrics for {image_path}:")
    print(f"PSNR: {psnr_value}")
    print(f"MSE: {mse_value}")
    print(f"SSIM: {ssim_value}")

    cv2.imshow('Original Image', original)
    cv2.imshow('Reconstructed Image - Contrast Enhancement', reconstructed)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Metrics for C:/Users/Joel Fredrick/Downloads/1.1.jpg:
PSNR: 14.06222440538522
MSE: 2551.8663216628333
SSIM: 0.7535412907600403
Metrics for C:/Users/Joel Fredrick/Downloads/1.2.jpg:
PSNR: 6.804040467691558
MSE: 13573.014532827981
SSIM: 0.4717212915420532
Metrics for C:/Users/Joel Fredrick/Downloads/1.3.jpg:
PSNR: 8.696824322258522
MSE: 8778.047541659307
SSIM: 0.40804240107536316


4. Laplacian Sharpening

In [5]:
def laplacian_sharpening(image):
    laplacian_kernel = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
    laplacian_filtered = cv2.filter2D(image, -1, laplacian_kernel)
    sharpened_image = image + laplacian_filtered
    return np.clip(sharpened_image, 0, 255)

def process_image(image_path):
    image = cv2.imread(image_path)
    original_shape = image.shape
    channels = cv2.split(image)
    enhanced_channels = []

    for channel in channels:
        if channel.shape[0] % 2 != 0:
            channel = channel[:-1, :]
        if channel.shape[1] % 2 != 0:
            channel = channel[:, :-1]

        coeffs = pywt.dwt2(channel, 'haar')
        LL, (LH, HL, HH) = coeffs

        enhanced_LL = laplacian_sharpening(LL)

        reconstructed_channel = pywt.idwt2((enhanced_LL, (LH, HL, HH)), 'haar')
        reconstructed_channel = reconstructed_channel[:channel.shape[0], :channel.shape[1]]
        reconstructed_channel = np.uint8(reconstructed_channel)
        enhanced_channels.append(reconstructed_channel)

    reconstructed_image = cv2.merge(enhanced_channels)
    reconstructed_image = cv2.resize(reconstructed_image, (original_shape[1], original_shape[0]))
    return image, reconstructed_image

def calculate_metrics(original, reconstructed):
    original = original.astype(np.float32)
    reconstructed = reconstructed.astype(np.float32)
    psnr_value = psnr(original, reconstructed, data_range=255)
    mse_value = mse(original, reconstructed)
    ssim_value = ssim(original, reconstructed, data_range=255, win_size=3, channel_axis=2)
    return psnr_value, mse_value, ssim_value

image_paths = [
    "C:/Users/Joel Fredrick/Downloads/1.1.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.2.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.3.jpg",
]

for image_path in image_paths:
    original, reconstructed = process_image(image_path)
    psnr_value, mse_value, ssim_value = calculate_metrics(original, reconstructed)

    print(f"Metrics for {image_path}:")
    print(f"PSNR: {psnr_value}")
    print(f"MSE: {mse_value}")
    print(f"SSIM: {ssim_value}")

    cv2.imshow('Original Image', original)
    cv2.imshow('Reconstructed Image - Laplacian Sharpening', reconstructed)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Metrics for C:/Users/Joel Fredrick/Downloads/1.1.jpg:
PSNR: 12.881852309394919
MSE: 3348.8456405997104
SSIM: 0.5095180869102478
Metrics for C:/Users/Joel Fredrick/Downloads/1.2.jpg:
PSNR: 9.731035791167095
MSE: 6917.9398524469025
SSIM: 0.5045284628868103
Metrics for C:/Users/Joel Fredrick/Downloads/1.3.jpg:
PSNR: 12.810010758567493
MSE: 3404.7034045810515
SSIM: 0.48657575249671936


Guided Image Filtering

In [3]:
import cv2
import numpy as np
import pywt
from skimage.metrics import peak_signal_noise_ratio as psnr, mean_squared_error as mse, structural_similarity as ssim

def guided_filtering(I, p, r=8, eps=0.01):
    I = I.astype(np.float32)
    p = p.astype(np.float32)
    
    mean_I = cv2.boxFilter(I, cv2.CV_32F, (r, r))
    mean_p = cv2.boxFilter(p, cv2.CV_32F, (r, r))
    mean_Ip = cv2.boxFilter(I * p, cv2.CV_32F, (r, r))
    
    cov_Ip = mean_Ip - mean_I * mean_p
    mean_II = cv2.boxFilter(I * I, cv2.CV_32F, (r, r))
    
    var_I = mean_II - mean_I * mean_I
    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I
    
    mean_a = cv2.boxFilter(a, cv2.CV_32F, (r, r))
    mean_b = cv2.boxFilter(b, cv2.CV_32F, (r, r))
    
    q = mean_a * I + mean_b
    return q

def process_image(image_path):
    image = cv2.imread(image_path)
    original_shape = image.shape
    channels = cv2.split(image)
    enhanced_channels = []

    for channel in channels:
        # Ensure the dimensions are even for DWT
        if channel.shape[0] % 2 != 0:
            channel = channel[:-1, :]
        if channel.shape[1] % 2 != 0:
            channel = channel[:, :-1]

        # Perform DWT
        coeffs = pywt.dwt2(channel, 'haar')
        LL, (LH, HL, HH) = coeffs

        # Apply guided filtering to the LL band
        enhanced_LL = guided_filtering(LL, LL)

        # Perform IDWT to reconstruct the image
        reconstructed_channel = pywt.idwt2((enhanced_LL, (LH, HL, HH)), 'haar')
        reconstructed_channel = np.clip(reconstructed_channel, 0, 255)
        reconstructed_channel = reconstructed_channel[:channel.shape[0], :channel.shape[1]]
        reconstructed_channel = np.uint8(reconstructed_channel)
        enhanced_channels.append(reconstructed_channel)

    # Merge the enhanced channels back into an image
    reconstructed_image = cv2.merge(enhanced_channels)
    reconstructed_image = cv2.resize(reconstructed_image, (original_shape[1], original_shape[0]))
    return image, reconstructed_image

def calculate_metrics(original, reconstructed):
    original = original.astype(np.float32)
    reconstructed = reconstructed.astype(np.float32)
    psnr_value = psnr(original, reconstructed, data_range=255)
    mse_value = mse(original, reconstructed)
    ssim_value = ssim(original, reconstructed, data_range=255, channel_axis=2)
    return psnr_value, mse_value, ssim_value

image_paths = [
    "C:/Users/Joel Fredrick/Downloads/1.1.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.2.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.3.jpg",
]

for image_path in image_paths:
    original, reconstructed = process_image(image_path)
    psnr_value, mse_value, ssim_value = calculate_metrics(original, reconstructed)

    print(f"Metrics for {image_path}:")
    print(f"PSNR: {psnr_value}")
    print(f"MSE: {mse_value}")
    print(f"SSIM: {ssim_value}")

    cv2.imshow('Original Image', original)
    cv2.imshow('Reconstructed Image - Guided Filtering', reconstructed)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Metrics for C:/Users/Joel Fredrick/Downloads/1.1.jpg:
PSNR: 51.25397062395283
MSE: 0.48717309821960986
SSIM: 0.9991269111633301
Metrics for C:/Users/Joel Fredrick/Downloads/1.2.jpg:
PSNR: 27.717428842966306
MSE: 109.9860174171454
SSIM: 0.9149708151817322
Metrics for C:/Users/Joel Fredrick/Downloads/1.3.jpg:
PSNR: 27.221234523646125
MSE: 123.29824883707236
SSIM: 0.8748995661735535


Total Variation (TV) Minimization

In [1]:
!pip install scikit-image


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\Joel Fredrick\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:

import cv2
import numpy as np
import pywt
from skimage.metrics import peak_signal_noise_ratio as psnr, mean_squared_error as mse, structural_similarity as ssim
from skimage.restoration import denoise_tv_chambolle

def apply_tv_minimization(image, weight=0.1):
    """Apply TV minimization to the image."""
    return denoise_tv_chambolle(image, weight=weight)

def process_image(image_path):
    image = cv2.imread(image_path)
    original_shape = image.shape
    channels = cv2.split(image)
    enhanced_channels = []

    for channel in channels:
        # Ensure the dimensions are even for DWT
        if channel.shape[0] % 2 != 0:
            channel = channel[:-1, :]
        if channel.shape[1] % 2 != 0:
            channel = channel[:, :-1]

        # Perform DWT
        coeffs = pywt.dwt2(channel, 'haar')
        LL, (LH, HL, HH) = coeffs

        # Apply TV minimization to the LL band
        enhanced_LL = apply_tv_minimization(LL)

        # Perform IDWT to reconstruct the image
        reconstructed_channel = pywt.idwt2((enhanced_LL, (LH, HL, HH)), 'haar')
        reconstructed_channel = np.clip(reconstructed_channel, 0, 255)
        reconstructed_channel = reconstructed_channel[:channel.shape[0], :channel.shape[1]]
        reconstructed_channel = np.uint8(reconstructed_channel)
        enhanced_channels.append(reconstructed_channel)

    # Merge the enhanced channels back into an image
    reconstructed_image = cv2.merge(enhanced_channels)
    reconstructed_image = cv2.resize(reconstructed_image, (original_shape[1], original_shape[0]))
    return image, reconstructed_image

def calculate_metrics(original, reconstructed):
    original = original.astype(np.float32)
    reconstructed = reconstructed.astype(np.float32)
    psnr_value = psnr(original, reconstructed, data_range=255)
    mse_value = mse(original, reconstructed)
    ssim_value = ssim(original, reconstructed, data_range=255, channel_axis=2)
    return psnr_value, mse_value, ssim_value

image_paths = [
    "C:/Users/Joel Fredrick/Downloads/1.1.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.2.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.3.jpg",
]

for image_path in image_paths:
    original, reconstructed = process_image(image_path)
    psnr_value, mse_value, ssim_value = calculate_metrics(original, reconstructed)

    print(f"Metrics for {image_path}:")
    print(f"PSNR: {psnr_value}")
    print(f"MSE: {mse_value}")
    print(f"SSIM: {ssim_value}")

    cv2.imshow('Original Image', original)
    cv2.imshow('Reconstructed Image - TV Minimization', reconstructed)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Metrics for C:/Users/Joel Fredrick/Downloads/1.1.jpg:
PSNR: 51.10991730314882
MSE: 0.5036033733708152
SSIM: 0.9986591935157776
Metrics for C:/Users/Joel Fredrick/Downloads/1.2.jpg:
PSNR: 27.754261693278224
MSE: 109.05716192701009
SSIM: 0.9153446555137634
Metrics for C:/Users/Joel Fredrick/Downloads/1.3.jpg:
PSNR: 27.265463548344258
MSE: 122.04893835011482
SSIM: 0.8758324980735779


TV minimization for LL band

In [2]:
import cv2
import numpy as np
import pywt
from skimage.metrics import peak_signal_noise_ratio as psnr, mean_squared_error as mse, structural_similarity as ssim
from skimage.restoration import denoise_tv_chambolle

def apply_tv_minimization(image, weight=0.1):
    """Apply TV minimization to the image."""
    return denoise_tv_chambolle(image, weight=weight)

def process_image(image_path):
    image = cv2.imread(image_path)
    original_shape = image.shape
    channels = cv2.split(image)
    enhanced_channels = []

    for channel in channels:
        if channel.shape[0] % 2 != 0:
            channel = channel[:-1, :]
        if channel.shape[1] % 2 != 0:
            channel = channel[:, :-1]

        coeffs = pywt.dwt2(channel, 'haar')
        LL, (LH, HL, HH) = coeffs

        # Apply TV minimization to the LL band
        enhanced_LL = apply_tv_minimization(LL)

        reconstructed_channel = pywt.idwt2((enhanced_LL, (LH, HL, HH)), 'haar')
        reconstructed_channel = np.clip(reconstructed_channel, 0, 255)
        reconstructed_channel = reconstructed_channel[:channel.shape[0], :channel.shape[1]]
        reconstructed_channel = np.uint8(reconstructed_channel)
        enhanced_channels.append(reconstructed_channel)

    reconstructed_image = cv2.merge(enhanced_channels)
    reconstructed_image = cv2.resize(reconstructed_image, (original_shape[1], original_shape[0]))
    return image, reconstructed_image

def calculate_metrics(original, reconstructed):
    original = original.astype(np.float32)
    reconstructed = reconstructed.astype(np.float32)
    psnr_value = psnr(original, reconstructed, data_range=255)
    mse_value = mse(original, reconstructed)
    ssim_value = ssim(original, reconstructed, data_range=255, channel_axis=2)
    return psnr_value, mse_value, ssim_value

image_paths = [
    "C:/Users/Joel Fredrick/Downloads/1.1.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.2.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.3.jpg",
]

for image_path in image_paths:
    original, reconstructed = process_image(image_path)
    psnr_value, mse_value, ssim_value = calculate_metrics(original, reconstructed)

    print(f"Metrics for {image_path}:")
    print(f"PSNR: {psnr_value}")
    print(f"MSE: {mse_value}")
    print(f"SSIM: {ssim_value}")

    cv2.imshow('Original Image', original)
    cv2.imshow('Reconstructed Image - TV Minimization', reconstructed)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Metrics for C:/Users/Joel Fredrick/Downloads/1.1.jpg:
PSNR: 51.10991730314882
MSE: 0.5036033733708152
SSIM: 0.9986591935157776
Metrics for C:/Users/Joel Fredrick/Downloads/1.2.jpg:
PSNR: 27.754261693278224
MSE: 109.05716192701009
SSIM: 0.9153446555137634
Metrics for C:/Users/Joel Fredrick/Downloads/1.3.jpg:
PSNR: 27.265463548344258
MSE: 122.04893835011482
SSIM: 0.8758324980735779


Guided Filtering for LL band

In [3]:
import cv2
import numpy as np
import pywt
from skimage.metrics import peak_signal_noise_ratio as psnr, mean_squared_error as mse, structural_similarity as ssim

def guided_filtering(I, p, r=8, eps=0.01):
    I = I.astype(np.float32)
    p = p.astype(np.float32)
    
    mean_I = cv2.boxFilter(I, cv2.CV_32F, (r, r))
    mean_p = cv2.boxFilter(p, cv2.CV_32F, (r, r))
    mean_Ip = cv2.boxFilter(I * p, cv2.CV_32F, (r, r))
    
    cov_Ip = mean_Ip - mean_I * mean_p
    mean_II = cv2.boxFilter(I * I, cv2.CV_32F, (r, r))
    
    var_I = mean_II - mean_I * mean_I
    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I
    
    mean_a = cv2.boxFilter(a, cv2.CV_32F, (r, r))
    mean_b = cv2.boxFilter(b, cv2.CV_32F, (r, r))
    
    q = mean_a * I + mean_b
    return q

def process_image(image_path):
    image = cv2.imread(image_path)
    original_shape = image.shape
    channels = cv2.split(image)
    enhanced_channels = []

    for channel in channels:
        if channel.shape[0] % 2 != 0:
            channel = channel[:-1, :]
        if channel.shape[1] % 2 != 0:
            channel = channel[:, :-1]

        # Perform DWT
        coeffs = pywt.dwt2(channel, 'haar')
        LL, (LH, HL, HH) = coeffs

        # Apply guided filtering to the LL band
        enhanced_LL = guided_filtering(LL, LL)

        # Zero out the high-frequency components
        zeroed_HL = np.zeros_like(HL)
        zeroed_LH = np.zeros_like(LH)
        zeroed_HH = np.zeros_like(HH)

        # Perform IDWT to reconstruct the image using only the enhanced LL band
        reconstructed_channel = pywt.idwt2((enhanced_LL, (zeroed_LH, zeroed_HL, zeroed_HH)), 'haar')
        reconstructed_channel = np.clip(reconstructed_channel, 0, 255)
        reconstructed_channel = reconstructed_channel[:channel.shape[0], :channel.shape[1]]
        reconstructed_channel = np.uint8(reconstructed_channel)
        enhanced_channels.append(reconstructed_channel)

    # Merge the enhanced channels back into an image
    reconstructed_image = cv2.merge(enhanced_channels)
    reconstructed_image = cv2.resize(reconstructed_image, (original_shape[1], original_shape[0]))
    return image, reconstructed_image

def calculate_metrics(original, reconstructed):
    original = original.astype(np.float32)
    reconstructed = reconstructed.astype(np.float32)
    psnr_value = psnr(original, reconstructed, data_range=255)
    mse_value = mse(original, reconstructed)
    ssim_value = ssim(original, reconstructed, data_range=255, channel_axis=2)
    return psnr_value, mse_value, ssim_value

image_paths = [
    "C:/Users/Joel Fredrick/Downloads/1.1.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.2.jpg",
    "C:/Users/Joel Fredrick/Downloads/1.3.jpg",
]

for image_path in image_paths:
    original, reconstructed = process_image(image_path)
    psnr_value, mse_value, ssim_value = calculate_metrics(original, reconstructed)

    print(f"Metrics for {image_path}:")
    print(f"PSNR: {psnr_value}")
    print(f"MSE: {mse_value}")
    print(f"SSIM: {ssim_value}")

    cv2.imshow('Original Image', original)
    cv2.imshow('Reconstructed Image - Guided Filtering on LL Band', reconstructed)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Metrics for C:/Users/Joel Fredrick/Downloads/1.1.jpg:
PSNR: 30.27634914000402
MSE: 61.01624073600818
SSIM: 0.9288592338562012
Metrics for C:/Users/Joel Fredrick/Downloads/1.2.jpg:
PSNR: 25.975554074916896
MSE: 164.25740347866162
SSIM: 0.8493617177009583
Metrics for C:/Users/Joel Fredrick/Downloads/1.3.jpg:
PSNR: 26.042921571534503
MSE: 161.72911264205382
SSIM: 0.8318047523498535
